#**Model tuning for Human Activity Recognition**

#Setup


*   Pytorch and flower installation

In [ ]:
!pip install torch torchvision opacus

In [ ]:
!pip install optuna

##All General Imports

In [ ]:
import os
import glob
import math
import json
import random
import timeit
import platform

from collections import OrderedDict
from hashlib import md5
from typing import Callable, Dict, List, Optional, Tuple, Union, NewType

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from opacus import PrivacyEngine
from opacus.accountants.rdp import RDPAccountant

In [ ]:
# Seaborn plot settings
sns.set_style("white")
palette = sns.color_palette("Set2")
sns.set_context("paper", font_scale=1.2)  # Increase font size

##All Machine Learning Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, TensorDataset
from torchvision.datasets import CIFAR10
from torch import Tensor

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from collections import Counter, OrderedDict

import optuna
from optuna.trial import TrialState

---
**Tested with flower version 3.3.0 and torch version 2.0.1+cu118**

---



In [ ]:
optuna.__version__

In [ ]:
torch.__version__

##Reproducibility Params

In [ ]:
# For dataloader workers
def _init_fn(worker_id):
    np.random.seed(int(random_seed))


def set_random_seeds(random_seed):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.use_deterministic_algorithms(True)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False

    torch.manual_seed(random_seed)

random_seed = 123
set_random_seeds(random_seed)


##All Globals

In [ ]:
experiment_params = {}

In [ ]:
# @title Globals { display-mode: "form" }
USE_DP = False # @param {type:"boolean"}
target_epsilon = 0.3 # @param {type:"number"}
# @markdown ---
experiment_params["USE_DP"] = USE_DP
if USE_DP:
  experiment_params["target_epsilon"] = target_epsilon

##Hyperparameters

In [ ]:
# @title ### Hypers { display-mode: "form" }
# @markdown ---
number_of_trials = 100 # @param {type:"slider", min:20, max:100, step:5}
timeout = 500 # @param {type:"slider", min:500, max:10000, step:500}
# @markdown ---
n_epochs = 15 # @param {type:"slider", min:1, max:25}
batch_size = 32 # @param {type:"slider", min:32, max:128, step:32}
test_split_size = 0.2 # @param {type:"slider", min:0.1, max:0.5}
# @markdown ---
learning_rate_start = .00001 # @param {type:"number"}
learning_rate_end = 0.01 # @param {type:"number"}
# @markdown ---
max_number_inner_layers = 2 # @param {type:"slider", min:1, max:5, step:1}
min_inner_size = 64 # @param {type:"slider", min:64, max:512, step:32}
max_inner_size = 512 # @param {type:"slider", min:64, max:512, step:32}
# @markdown ---
#
experiment_params["number_of_trials"] = number_of_trials
experiment_params["timeout"] = timeout
#
experiment_params["n_epochs"] = n_epochs
experiment_params["batch_size"] = batch_size
experiment_params["test_split_size"] = test_split_size
#
experiment_params["learning_rate_start"] = learning_rate_start
experiment_params["learning_rate_end"] = learning_rate_end
#
experiment_params["max_number_inner_layers"] = max_number_inner_layers
experiment_params["min_inner_size"] = min_inner_size
experiment_params["max_inner_size"] = max_inner_size

##Initializations

In [ ]:
experiment_json = json.dumps(experiment_params)

In [ ]:
# @title Save path
save_path = md5(experiment_json.encode()).hexdigest()[:8]
print(save_path)

In [ ]:
with open(f'{save_path}_optuna.json', 'w') as f:
    f.write(experiment_json)

In [ ]:
start_global_time = timeit.default_timer()

if not os.path.exists(save_path):
    os.makedirs(save_path)

with open(f'{save_path}/optuna_parameters.json', 'w') as f:
    f.write(experiment_json)

DEVICE = torch.device("cpu")  # Prova "cuda" per addestramento su GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Optuna {optuna.__version__}"
)

OS = platform.system()           # Sistema Operativo

#Data preparation

##Data Download

In [ ]:
def data_download(file_to_download, gdrive_code, OS, uncompress = True):
  if not os.path.exists(file_to_download):
    os.system('gdown --id "'+gdrive_code+'" --output '+file_to_download)
    if OS == "Linux" and uncompress:
        os.system('unzip -o -n "./'+file_to_download+'" -d '+os.path.dirname(file_to_download))
    return True
  else:
    return None



In [ ]:
out = data_download("./har_datasets_fl.zip", "1LUjU4yvBRh6FPBlIHRCD2uf5zMH6l9tC", OS)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip", filename="har-data.zip")


In [ ]:
trainloaders = []

# Awful hack, when True this flips test and train datasets for a stratified
# split ensuring that independent balanced samples are distributed in each split
# Normal behavior flip=False
flip = False
def stratified_split(data, targets, n_splits, split_size=None):
    # NOTE: We pick one stratified split => n_splits=1 because we want a
    # balanced test set, the training part will be postprocessed
    if not split_size:
      df = pd.DataFrame(data)
      data_length = len(df)
      split_size = int(data_length / n_splits)
      print("split_size", test_size)
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=split_size, random_state=random_seed)
    for train_index, val_index in sss.split(data, targets):
        if flip:
          yield data[val_index], targets[val_index], data[train_index], targets[train_index]
        else:
          yield data[train_index], targets[train_index], data[val_index], targets[val_index]

def gini_index(y):
  uniques = np.unique(y+1, return_counts=True)
  probs = uniques[1]/np.sum(uniques[1])
  #print(uniques, probs, np.sum(probs))
  gini_index = 1.0 - np.sum(probs ** 2)
  return gini_index

def get_data_from_path(path):
    fold_number = os.path.basename(path).split('-')[0].strip()
    trainset = pd.read_csv(f"{path}/train/{fold_number}_ALL_train.csv", delimiter=';')
    testset = pd.read_csv(f"{path}/test/{fold_number}_ALL_test.csv", delimiter=';')
    return trainset, testset

def create_datasets_from_dataframe(df):
    # Extract features from columns '0' to '560'
    X = pd.concat([df[str(i)] for i in range(561)], axis=1).values
    # Adjust labels in 'Y' column to start from 0
    y = (df['Y'] - 1).values

    return X, y

In [ ]:
# Let's combine the old data splits into a single dataframe
all_data = []
for path in [f.path for f in os.scandir('./har_datasets_fl') if f.is_dir()]:
    train_df, test_df = get_data_from_path(path)
    all_data.append(train_df)
    all_data.append(test_df)

combined_df = pd.concat(all_data, axis=0)
print(f"Total data points {len(combined_df)}")

X_all, y_all = create_datasets_from_dataframe(combined_df)

# 1st stratified to get all train data and test data for (server) evaluation
X_train_combined, y_train_combined, X_test, y_test =\
  next(stratified_split(X_all, y_all, n_splits=1, split_size=test_split_size))

print(f"Total train data points {len(X_train_combined)}")
print(f"Total test data points {len(X_test)}")

In [ ]:
def generate_dataloaders(data, targets):
    dataloaders = []
    # Assuming set_random_seeds function is defined elsewhere
    set_random_seeds(random_seed)

    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=random_seed)

    for train_index, test_index in sss.split(data, targets):
        train_dataset = TensorDataset(torch.tensor(data[train_index]).float(), torch.tensor(targets[train_index]).long())
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_dataset = TensorDataset(torch.tensor(data[test_index]).float(), torch.tensor(targets[test_index]).long())
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_loader, test_loader

In [ ]:
train_loader, test_loader = \
  generate_dataloaders(X_test, y_test)

In [ ]:
# number of classes
n_classes = len(np.unique(y_train_combined))
# Assuming class_names is a dictionary mapping class numbers to class names
class_names = {0: "Walking", 1: "Walking\nupstairs", 2: "Walking\ndownstairs", 3: "Sitting", 4: "Standing", 5: "Laying"}

In [ ]:
# plot classes distribution

#Model

In [ ]:
def define_model(trial):
    # We optimize the number of layers
    n_layers = trial.suggest_int("n_layers", 1, max_number_inner_layers)
    layers = []
    in_features = 561
    for i in range(n_layers):
      out_features = trial.suggest_int("n_units_innerlayer_{}".format(i), min_inner_size, max_inner_size)
      layers.append(nn.Linear(in_features, out_features))
      layers.append(nn.ReLU())
      in_features = out_features
    layers.append(nn.Linear(in_features, n_classes))
    return nn.Sequential(*layers)

In [ ]:
class MLP(nn.Module):
    """ Multi Layer Perceptron """
    def __init__(self) -> None:
        super(MLP, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(561, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 6)
        )

    def forward(self, x: Tensor) -> Tensor:
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Net = MLP

##Training

###Parameter updates

###Training function

In [ ]:
def train(model, trainloader, epochs: int, optimizer):
    torch.manual_seed(random_seed)
    torch.use_deterministic_algorithms(True)

    training_size = len(trainloader.dataset)
    batch_size = trainloader.batch_size

    # Modify target_epsilon and target_delta here
    noise_generator = torch.Generator()
    noise_generator.manual_seed(random_seed)

    target_delta = 1e-5

    max_grad_norm = 1.0
    noise_multiplier = 1.0  # This value will be used to initialize the PrivacyEngine, but it will be modified automatically to reach the target epsilon

    criterion = torch.nn.CrossEntropyLoss()
    #optimizer = torch.optim.Adam(net.parameters())

    dataloader = trainloader  # Define dataloader here

    if USE_DP:
        privacy_engine = PrivacyEngine(accountant = 'rdp')

        model, optimizer, dataloader = privacy_engine.make_private_with_epsilon(
            module=model,
            optimizer=optimizer,
            data_loader=dataloader,
            target_epsilon=target_epsilon,
            target_delta=target_delta,
            epochs=epochs,
            max_grad_norm=max_grad_norm,
            noise_generator=noise_generator
        )
    else:
        # If not using DP, PrivacyEngine is not defined and can't be used to get epsilon later.
        privacy_engine = None

    #model = model.to(DEVICE)

    model.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in dataloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()

        epoch_loss /= len(dataloader.dataset)

    # After training, you can get the final epsilon
    if privacy_engine:  # Only try to get epsilon if privacy_engine was defined
        final_epsilon = privacy_engine.get_epsilon(delta=target_delta)
        print(f"The target epsilon was: {target_epsilon}")
        print(f"The final epsilon is: {final_epsilon}")


###Model Testing

In [ ]:
def test(net, testloader):
    """Evaluate the network on the entire test set."""

    torch.manual_seed(random_seed)
    torch.use_deterministic_algorithms(True)
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()

    all_labels = []
    all_predicted = []

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.append(labels.cpu())
            all_predicted.append(predicted.cpu())

    all_labels = torch.cat(all_labels) # concatenate all labels tensors
    all_predicted = torch.cat(all_predicted) # concatenate all predicted tensors

    loss /= len(testloader.dataset)
    accuracy = correct / total

    # Calculate F1 score. Need to convert tensors to numpy arrays
    f1_score_value_micro = f1_score(all_labels.numpy(), all_predicted.numpy(), average='micro')
    f1_score_value_macro = f1_score(all_labels.numpy(), all_predicted.numpy(), average='macro')
    f1_score_value_perclass = f1_score(all_labels.numpy(), all_predicted.numpy(), average=None)

    return accuracy, loss, f1_score_value_micro, f1_score_value_macro, f1_score_value_perclass


In [ ]:
def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    #lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    lr = trial.suggest_float("lr", learning_rate_start, learning_rate_end, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train(model, train_loader, n_epochs, optimizer)
    accuracy, loss, f1_score_value_micro, f1_score_value_macro, f1_score_value_perclass = test(model, test_loader)
    return accuracy, f1_score_value_macro

In [ ]:
%%time
study = optuna.create_study(directions=["minimize", "maximize"])
study.optimize(objective, n_trials=number_of_trials, timeout=timeout)

print("Number of finished trials: ", len(study.trials))

In [ ]:
    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

In [ ]:
trial_with_highest_accuracy = max(study.best_trials, key=lambda t: t.values[0])
print(f"Trial with highest accuracy: ")
print(f"\tnumber: {trial_with_highest_accuracy.number}")
print(f"\tparams: {trial_with_highest_accuracy.params}")
print(f"\tvalues: {trial_with_highest_accuracy.values}")

In [ ]:
trial_with_highest_f1_macro = max(study.best_trials, key=lambda t: t.values[1])
print(f"Trial with highest f1_macro: ")
print(f"\tnumber: {trial_with_highest_f1_macro.number}")
print(f"\tparams: {trial_with_highest_f1_macro.params}")
print(f"\tvalues: {trial_with_highest_f1_macro.values}")

In [ ]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.values[0], target_name="accuracy"
)

In [ ]:
print(f"Elapsed time {timeit.default_timer()- start_global_time}")

In [ ]:
experiment_params

From this search

These are the hypers to keep
```
{'USE_DP': False,
 'number_of_trials': 100,
 'timeout': 500,
 'n_epochs': 15,
 'batch_size': 32,
 'test_split_size': 0.2,
 'learning_rate_start': 1e-05,
 'learning_rate_end': 0.01,
 'max_number_inner_layers': 2,
 'min_inner_size': 64,
 'max_inner_size': 512}
```

Trial with highest f1_macro:
```
  number: 84
	params: {'n_layers': 2, 'n_units_innerlayer_0': 437, 'n_units_innerlayer_1': 312, 'optimizer': 'Adam', 'lr': 0.0018673528886359607}
	values: [0.9635922330097088, 0.9650936723136906]
  ```


In [ ]:
trials_df = []
for ct in complete_trials:
  trials_df.append([ct.number, ct.values[0], ct.values[1], ct.params['n_layers']])


In [ ]:
trials_df = pd.DataFrame(trials_df, columns=["number", "acc", "f1", "n_layers"])

In [ ]:
trials_df

In [ ]:
best_one_layer = trials_df[trials_df["n_layers"]==1].sort_values(by=["f1","acc"], ascending=False)[:3]
best_one_layer

In [ ]:
for i in list(best_one_layer.index):
  ct = complete_trials[i]
  print(ct.params)

Best with one inner layer:
```
{'n_layers': 1, 'n_units_innerlayer_0': 437, 'optimizer': 'Adam', 'lr': 0.0013429456755218343}
{'n_layers': 1, 'n_units_innerlayer_0': 249, 'optimizer': 'Adam', 'lr': 0.001183653983362325}
{'n_layers': 1, 'n_units_innerlayer_0': 437, 'optimizer': 'Adam', 'lr': 0.00034193430144998076}
```
